<img src="assets/jupyter-micropython-esp32-2.png">


***
# MicroPython fun on €10 hardware


[MicroPython](https://github.com/micropython/micropython) is an implementation of the Python 3 optimised to run on microcontrollers.
- MicroPython started as a Kickstarter campaign in 2013 by Damien George.
- MicroPython is a full Python compiler and runtime that runs on the microcontroller hardware.
- Compilation is on the chip so no need to install tools on your laptop.
- Can interact with the the REPL to run code directly on the hardware. 
- Includes a subset of core Python libraries (most beginning with 'u'). Hardware specific libraries that vary for each hardware port.
- If you don't have a microcontroller try out the [online emulator](http://micropython.org/unicorn/)



MicroPython downloads and ESP32 installation instructions are available [here](https://micropython.org/download#esp32).


***
# Jupyter Notebook MicroPython kernel install

We are going to be using the [Jupyter MicroPython Remote kernel](https://pypi.org/project/jupyter-micropython-remote/) to interact with a MicroPython board over its REPL interface. 



From the shell command to install the MicroPython kernel for Jupyter run:

```
pip install jupyter_micropython_remote
```

Next register the kernel with Jupyter:

```
python -m mpy_kernel.install
```
    
And that should be it!!! Run Jupyter notebooks:

```
jupyter notebook
```

You should now have a **MicroPython Remote** option when creating a new Jupyter notebook.



***
# What does €10 get you?


<img src="assets/esp32_5_10_15.png">



Quite a lot actually:
- €5  ESP32 with 4MB flash, Wi-Fi & Bluetooth, lots of I/O.
- €10 ESP32 with 16MB flash, 8MB PSRAM, Wi-Fi & Bluetooth, lots of I/O, lithium battery charging circuit, SD card slot.
- €15 ESP32 with 4MB flash, Wi-Fi & Bluetooth, lots of I/O, lithium battery charging circuit, OLED display, LoRa wireless with km range.

ESP32 developer boards typically consist of:
- ESP32 System On Chip (SOC) as seen at top of the first two of these boards.
- USB interface for power and primary communication during development as seen at the bottom of these boards.
- GPIO and power pins exposes on two rows of pins either side of the board.
- Various manufacturers provide additional features like battery charging, display, camera, LoRa wireless, GPS on their boards.


<img src="assets/esp32-block-diagram.jpg">



The ESP32 SOC:
- Dual core, clock frequency up to 240MHz, 512 KB internal RAM.
- Modules come in different variants with additional external flash and PSRAM.
- Runs 32 bit programs.
- Wi-Fi and bluetooth built-in.
- Up to 34 GPIO pins.
- Ultra low power co-processor with access to GPIO during deepsleep.
- Built in security and encrytion hardware.



Check available documentation before buying. 
- Are board schematics available (usually based on an Espressif reference design)? 
- Are common parts used for display, LoRa chips, etc? There is better library support for common components.


***
# Establish a notebook connection to your microcontroller 

The first cell of your Jupyter Notebook should contain something like this:

```
%connect <device> --baudrate=115200 --user='micro' --password='python' --wait=0
```

You may need to reset the microcontroller with a soft reboot:

```
%reboot
```

**Note: Remember that you should only have one connection open on the serial port connected to the hardware.**



In [1]:
%connect COM4 --baudrate=115200

Connected on COM4


In [1]:
# Lets look at some of the microcontroller system details

import sys

print("Platform: {}".format(sys.platform))
print("MicroPython Version: {}".format(sys.implementation))


Platform: esp32
MicroPython Version: (name='micropython', version=(1, 11, 0))


In [ ]:
# List of magic functions.

%lsmagic


***
# MicroPython modules

Lets see what MicroPython libraries are available on the ESP32 board:

- [MicroPython standard libraries and micro-libraries](https://docs.micropython.org/en/latest/library/index.html#python-standard-libraries-and-micro-libraries)




In [1]:
# Get a list of modules available for the ESP32 port.

help('modules')

__main__          framebuf          socket            upip
_boot             gc                ssl               upip_utarfile
_onewire          hashlib           struct            upysh
_thread           heapq             sys               urandom
_webrepl          inisetup          time              ure
apa106            io                ubinascii         urequests
array             json              ucollections      uselect
binascii          machine           ucryptolib        usocket
btree             math              uctypes           ussl
builtins          micropython       uerrno            ustruct
cmath             neopixel          uhashlib          utime
collections       network           uhashlib          utimeq
dht               ntptime           uheapq            uwebsocket
ds18x20           onewire           uio               uzlib
errno             os                ujson             webrepl
esp               random            umqtt/robust      webrepl_setup
esp32   

In [1]:
# The machine module` contains functions related to the hardware on the board. 
# Including: I2C, SPI, UART interface, etc

import machine

help(machine)

object <module 'umachine'> is of type module
  __name__ -- umachine
  mem8 -- <8-bit memory>
  mem16 -- <16-bit memory>
  mem32 -- <32-bit memory>
  freq -- <function>
  reset -- <function>
  unique_id -- <function>
  sleep -- <function>
  lightsleep -- <function>
  deepsleep -- <function>
  idle -- <function>
  disable_irq -- <function>
  enable_irq -- <function>
  time_pulse_us -- <function>
  Timer -- <class 'Timer'>
  WDT -- <class 'WDT'>
  SDCard -- <class 'SDCard'>
  SLEEP -- 2
  DEEPSLEEP -- 4
  Pin -- <class 'Pin'>
  Signal -- <class 'Signal'>
  TouchPad -- <class 'TouchPad'>
  ADC -- <class 'ADC'>
  DAC -- <class 'DAC'>
  I2C -- <class 'I2C'>
  PWM -- <class 'PWM'>
  RTC -- <class 'RTC'>
  SPI -- <class 'SoftSPI'>
  UART -- <class 'UART'>
  reset_cause -- <function>
  HARD_RESET -- 2
  PWRON_RESET -- 1
  WDT_RESET -- 3
  DEEPSLEEP_RESET -- 4
  SOFT_RESET -- 5
  wake_reason -- <function>
  PIN_WAKE -- 2
  EXT0_WAKE -- 2
  EXT1_WAKE -- 3
  TIMER_WAKE -- 4
  TOUCHPAD_WAKE -- 5
  

***
# Controlling machine hardware

Lets work through some examples of controlling hardware with MicroPython.


<img src="assets/esp32_pinout.png">


Peripheral Input/Output include:
- 2 × I²C (Inter-Integrated Circuit).
- 3 x UART (universal asynchronous receiver/transmitter).
- 4 × SPI (Serial Peripheral Interface).
- 2 × I²S (Integrated Inter-IC Sound).
- CAN 2.0 (Controller Area Network).
- PWM (pulse width modulation) up to 16 channels. Useful for LED or motor control.
- 10 x Capacitive touch sensors.
- 12-bit ADCs (analog-to-digital converter) up to 18 channels.
- 2 × 8 bit DACs (digital-to-analog converter).
- Hall sensor.
- Internal temperature sensor.



***
## Blink the 'Hello World' for microcontroller

Lets set a [GPIO pin](https://docs.micropython.org/en/latest/library/machine.Pin.html) as an output and use it to drive an LED. The ESP32 board I'm using has an onboard LED wired to GPIO pin 26.


In [1]:
# Set GPIO pin as output and toggle state.

from machine import Pin
import time

# The Geekworm board has two onboard LEDs and a user button:
# LEDs: Pin 26
# Button: Pin 0

led = Pin(26, Pin.OUT)
led.off()

print('Blink LED', end= ' ')

for _ in range(10):
    led.value(not led.value())
    time.sleep_ms(1000)
    print('.', end= ' ')
    


Blink LED . . . . . . . . . . 

***
## Pulse Width Modulation (PWM)

PWM is typically used to control the brightness of a LED or to control motor speed.

To use PWM first create a Pin object and then specify the pulse frequency and duty cycle.
- ESP32 can generate PWM on most I/O pins.
- Frequency is from 1Hz to several MHz.
- Duty cycle is between 0 (all off) and 1023 (all on), with 512 being 50% duty.



In [1]:
# Demo: Controller LED with PWM.

from machine import Pin, PWM

led_pwm = PWM(Pin(26), freq=1, duty=512)

print('Frequency: {} Hz'.format(led_pwm.freq()))
print('Duty Cycle: {:.1%}'.format(led_pwm.duty()/1024))

Frequency: 1 Hz
Duty Cycle: 50.0%


In [1]:
# Deactivate PWM

led_pwm.deinit()

***
## Hardware Timer

To use a [hardware timer](https://docs.micropython.org/en/latest/library/machine.Timer.html) select a timer id, the trigger mode, and specify a callback function.
- The ESP32 has 4 hardware timers. Can also specify a software timer by using id = -1
- Timer period in ms.
- Timer mode:
 - ```Timer.ONE_SHOT``` - The timer runs once until the configured period of the channel expires.
 - ```Timer.PERIODIC``` - The timer runs periodically at the configured frequency of the channel.
- Callback function is called when triggered.



In [1]:
# Control a LED with a Timer.

from machine import Pin, Timer

led = Pin(26, Pin.OUT)

def flash_led(timer):
    led.value(not led.value())
    
# Use hardware timer 0
flash_timer = Timer(0)
flash_timer.init(period=1000, mode=Timer.PERIODIC, callback=flash_led)


In [1]:
# Disable timer.

flash_timer.deinit()

***
## Hardware Interrupt

A [hardware interrupt](https://docs.micropython.org/en/latest/library/machine.Pin.html) occurs in response to an external event, eg. a GPIO Interrupt (when a key is pressed).

- You can use all GPIOs as interrupts, except GPIO 6 to GPIO 11 (reserved for external flash memory interface).
- Set a callback function to be executed on the trigger. Callback functions should be as short and simple as possible.
- Interrupts can be triggers on rising/falling edge or pin level.
- Option to specify the power mode in which to *wake* up the system from sleep. It can be:
 - ```machine.IDLE```
 - ```machine.SLEEP```
 - ```machine.DEEPSLEEP```


In [1]:
# Trigger a hardware interrupt from a button press and toggle a LED.

from machine import Pin

# Geekworm board has two onboard LEDs and a user button:
# LEDs: Pin 26
# Button: Pin 0

led = Pin(26, Pin.OUT)
button = Pin(0, Pin.IN)

def button_isr(pin):
    led.value(not led.value())

button.irq(trigger=Pin.IRQ_FALLING, handler=button_isr)


In [1]:
# Disable IRQ

button.irq(trigger=0)

***
# NeoPixel

[Neopixel](https://docs.micropython.org/en/latest/esp8266/tutorial/neopixel.html), also known as WS2812 LEDs, are individually addressable RGB LED strips 

<img src="assets/neopixel.png">


In [1]:
# Configure a NeoPixel strip of 24 LEDs controlled by pin 26.

from machine import Pin
import neopixel, time

n = 24    # Neopixel led count
led_pin = 26

np = neopixel.NeoPixel(Pin(led_pin, Pin.OUT), n)


def cycle(count):
    for i in range(count * n):
        for j in range(n):
            np[j] = (0, 0, 0)
        np[i % n] = (0, 64, 0)
        np.write()
        time.sleep_ms(250)


# Cycle NeoPixels for 2 loops
cycle(2)


***
# %local cell execution


Individual cells can also be run on the local PC instead of the remote kernel by starting a cell with ```%local```

In %local cells, a special global function ```remote()``` is available which will pass a single string argument to the micropython board to be run.

This can be useful to work directly with local files, use [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html), etc. 




In [1]:
# MicroPython cell executed on remote hardware

from machine import Pin
import neopixel

n = 24    # Neopixel led count
led_pin = 26
    
np = neopixel.NeoPixel(Pin(led_pin, Pin.OUT), n)

def set_colour(r, g, b):
    for i in range(n):
        np[i] = (r, g, b)
    np.write()

# Set default LED level
set_colour(0x00, 0x00, 0x00)

In [1]:
# Run a colour picker locally

%local

from ipywidgets import interact, ColorPicker

def set_led(led_colour):
    r = int(led_colour[1:3], 16)
    g = int(led_colour[3:5], 16)
    b = int(led_colour[5:7], 16)

    remote(f"set_colour({r}, {g}, {b})")

    print("R:{} G:{} B:{}".format(r, g, b))


pick_colour = ColorPicker(description='LED Colour', value='#000000')
interact(set_led, led_colour=pick_colour)



interactive(children=(ColorPicker(value='#000000', description='LED Colour'), Output()), _dom_classes=('widget…

<function __main__.set_led(led_colour)>

***
# I2C bus & sensors

<img src="assets/I2C_bus.png">

The ESP32 offers support for a wide variety of peripheral interfaces with one of the most popular being Inter-Integrated Circuit (I²C or I2C). I2C uses two wires to communicate between the bus master and slave devices: a bi-directional serial data wire (SDA), and clock line (SCL). The bus supports multiple masters and up to 117 slaves using the 7 bit address scheme.

I2C is a popular interface for slower devices (Standard mode (100 Kbit/s), Fast mode (400 Kbit/s)) as it is cheap and requires minimum wiring for the device (typically VCC, GND, SDA, SCL).




***
## I2C bus scan

Each I2C slave has a unique address. A scanner cycles through all 127 possible slave device addresses, and checking whether or not an acknowledge is received.




In [2]:
from machine import I2C, Pin

i2c = I2C(scl=Pin(22), sda=Pin(21))

print('Scan i2c bus...')
devices = i2c.scan()

if len(devices) == 0:
    print('No i2c device !')
else:
    print('i2c devices found:', len(devices))
    for device in devices:
        print('Decimal address: ', device, ' | Hexa address: ', hex(device))


Scan i2c bus...
i2c devices found: 1
Decimal address:  118  | Hexa address:  0x76


***
## BME280 environmental sensor

<img src="assets/bme280.jpg">

The BME280 sensor measures temperature, humidity, and barometric pressure.
- I2C address: 0x76
- [BME280 tutorial with MicroPython library](https://randomnerdtutorials.com/micropython-bme280-esp32-esp8266/). I have already uploaded the Micropython BME280 module to the board flash in directory ```/lib```.


MicroPython looks for modules using ```sys.path``` which defaults to ```['', '/lib']```



In [2]:
from machine import Pin, I2C
from time import sleep
import BME280

# ESP32 - Pin assignment
i2c = I2C(scl=Pin(22), sda=Pin(21), freq=10000)

for _ in range(3):
    bme = BME280.BME280(i2c=i2c)
    temp = bme.temperature
    hum = bme.humidity
    pres = bme.pressure

    print('Temperature: ', temp)
    print('Humidity: ', hum)
    print('Pressure: ', pres)
    
    sleep(5)


Temperature:  20.18C
Humidity:  61.32%
Pressure:  1001.39hPa
Temperature:  20.13C
Humidity:  61.26%
Pressure:  1001.39hPa
Temperature:  20.14C
Humidity:  61.23%
Pressure:  1001.33hPa


***
# Wireless connectivity

The ESP32 offers a cheap solution for IoT projects with Bluetooth and WiFi. Wireless protocols that are supported:
- Wi-Fi: 802.11 b/g/n.
- Bluetooth: v4.2 and BLE.


Can set up an ESP32 as a WiFi Station ```network.STA_IF``` or Access Point ```network.AP_IF```.


In [ ]:
# Scan for available WiFi networks.

import network

wlan = network.WLAN(network.STA_IF)
wlan.active(True)

nets = wlan.scan()
for n in nets:
    # WiFi: (ssid, bssid, channel, RSSI, authmode, hidden). 
    print('ssid: {}, bssid: {}, channel: {}, RSSI: {}, authmode: {}, hidden: {}'.format(n[0],n[1],n[2],n[3],n[4],n[5]))

wlan.disconnect()

***
## Accessing PC filesystem from the microcontroller

Your local PC's working Jupyter directory is mounted at directory ```/remote/``` on the remote micropython hardware. This allows you to view, open, read, write and copy files to and from the microcontroller to your PC.


In [2]:
# Load WiFi credentials from config file stored in the PC Jupyter working directory.

import json, os

print(os.listdir('/'))

try:
    with open('/remote/assets/test-config.json', 'r') as f:
        data = json.load(f)
        ssid = data['ssid']
        wifi_pwd = data['wifi_pwd']
        
except FileNotFoundError:
    ssid = "your_ssid"
    wifi_pwd = "your_pwd"


#print('SSID: {}'.format(ssid))
#print('WiFi pwd: {}'.format(wifi_pwd))


['remote', 'boot.py', 'lib', 'main.py']


In [2]:
# Connect ESP32 to local WiFi network using credentials read from a JSON configuration file above.

import network

# Get WiFi credentials in cell above

wlan = network.WLAN(network.STA_IF)

if not wlan.isconnected():
    print('connecting to network...')
    wlan.active(True)
    wlan.connect(ssid, wifi_pwd)
    while not wlan.isconnected():
        pass

print('Network config:', wlan.ifconfig())

#wlan.disconnect()

connecting to network...
Network config: ('192.168.1.179', '255.255.255.0', '192.168.1.254', '192.168.1.254')


***
## Set real time clock from NTP server

The ESP32 has an internal [RTC](http://docs.micropython.org/en/latest/library/machine.RTC.html) however it is not very accurate. If you need accurate time measurement over an extended period use an external RTC like the DS3231.

Since we have wireless conectivity we can sychronise the ESP32 clock with an internet NTP server.



In [2]:
# Connect to NTP server and set RTC

import ntptime, machine

rtc = machine.RTC()
print('Before NTP sync: {}'.format(rtc.datetime()))

#ntptime.host = 'pool.ntp.org'    # Default host used if not set
ntptime.host = 'time.google.com'

ntptime.settime()

print('After NTP sync:  {}'.format(rtc.datetime()))


Before NTP sync: (2000, 1, 1, 5, 0, 6, 38, 263770)
After NTP sync:  (2019, 10, 10, 3, 18, 16, 32, 258)


***
## HTTP requests

Micropython HTTP library is ```urequests``` and has support for:
- SSL
- Cookies
- Basic Auth
- Custom HTTP Headers
- GET, POST, PUT, PATCH, DELETE, HEAD




In [2]:
import urequests

help(urequests)


object <module 'urequests' from 'urequests.py'> is of type module
  put -- <function put at 0x3ffcc890>
  post -- <function post at 0x3ffcc880>
  usocket -- <module 'usocket'>
  patch -- <function patch at 0x3ffcc8a0>
  request -- <function request at 0x3ffcc860>
  __file__ -- urequests.py
  __name__ -- urequests
  delete -- <function delete at 0x3ffcc8b0>
  head -- <function head at 0x3ffcc6e0>
  Response -- <class 'Response'>
  get -- <function get at 0x3ffcc6f0>


In [2]:
import urequests

response = urequests.get('http://jsonplaceholder.typicode.com/todos/22')

print(response.text)

# Since the response is of type JSON, we can deserialise into a dictionary for easier parsing
print('\nResponse as dictionary: {}'.format(response.json()))

print('\nStatus Code: {}'.format(response.status_code))

print('\nReason: {}'.format(response.reason.decode('utf-8')))

response.close()


{
  "userId": 2,
  "id": 22,
  "title": "distinctio vitae autem nihil ut molestias quo",
  "completed": true
}

Response as dictionary: {'id': 22, 'userId': 2, 'title': 'distinctio vitae autem nihil ut molestias quo', 'completed': True}

Status Code: 200

Reason: OK


***
# ESP32 deep sleep

The microcontroller can enter a sleep state which stops execution in an attempt to enter a low power state.
- ```machine.lightsleep```: A lightsleep has full RAM and state retention. Upon wake execution is resumed from the point where the sleep was requested, with all subsystems operational.
- ```machine.deepsleep```: A deepsleep may not retain RAM or any other state of the system (date can be stored in the RTC module memory). Upon wake execution is resumed from the main script, similar to a hard or power-on reset.

ESP32 can wake up after sleeping for a specified time. If no time interval is specified sleep can last indefinitely or until a **wake interrupt** is received from one of the following sources:
- Timer.
- External wake up when a change in the state of a pin occurs.
- Touch pins.


In [ ]:
import machine
import esp32
from machine import Pin
from time import sleep

wake1 = Pin(0, mode = Pin.IN)
wake2 = Pin(34, mode = Pin.IN)

esp32.wake_on_ext1(pins = (wake1, wake2), level = esp32.WAKEUP_ALL_LOW)

for i in reversed(range(5)):
    print('Going to sleep in {} seconds...'.format(i+1))
    sleep(1)

print('Going to sleep now')


# Deep sleep until external wake up pin pressed
machine.deepsleep()


# To get the reason for the wake event use machine.wake_reason()
# Possible wakeup_reason values:
#    0 - no wake-up reason
#    1 - EXT_0 wake-up
#    2 - EXT_1 wake-up
#    3 - Touchpad wake-up
#    4 - RTC wake-up
#    5 - ULP wake-up

machine.wake_reason()


In [3]:
# Try to reconnect after waking from deepsleep

%connect COM4 --baudrate=115200

Connected on COM4


***
# Uploading your Python script to the board

With MicroPython boards you can add as many scripts as you like. You are only limited by the size of available flash memory. The file system contains two important files:
1. ```/boot.py``` is run when the board is powered on or reset. Generally it is not edited and should only be used to setup board configuration.
2. ```/main.py``` is an optional file and is run after boot.py. This file can be used to start your application script after power on. 



***
## rshell

[Remote MicroPython shell: rshell](https://github.com/dhylands/rshell)


To connect rshell to the MicroPytohn hardware:

```
rshell -p COM4
```

The should establish a connection to the board. rshell assumes your board is called **/pyboard/**

```
ls /pyboard -l
```

To enter the REPL on the hardware type **repl**. **help()** to display REPL help. Ctrl + X to exit the REPL.

To copy file to the ESP32 from your local directory:

```
cp *.py /pyboard
ls /pyboard
```



In [ ]:
# The os module contains functions for filesystem access.

import os

help(os)

In [3]:

print('\n'.join(os.listdir('/')))


remote
boot.py
lib
main.py


In [ ]:
# Disconnect notebook from board before using rshell

%disconnect

***
## VS Code & Pymakr

A recommendation on a MicroPython IDE is VS Code with the [Pymakr extension](https://docs.pycom.io/pymakr/installation/vscode/). An overview of the extension usage and recommended project structure is [First PyMakr project](https://docs.pycom.io/gettingstarted/programming/first-project/)

The Pymakr extension features:
- Connects to remote MicroPython board.
- Interact with the board with the command line REPL.
- Upload your project to the board.
- Download files from the board.
- Run current editor file or select lines of code.



***
# Debugging with Sigrok PulseView & logic analyzers

<img src="assets/LogicAnalyzer.png">


You may find it necessary to debug your hardware projects and the combination of [Sigrok PulseView](https://sigrok.org/doc/pulseview/0.4.1/manual.html) and cheap USB logic analyzer hardware (€10 to €20) is a great addition to your toolkit.

One of the nice features of PulseView is [protocol decoders](https://sigrok.org/wiki/Protocol_decoders) which can be stacked to decode signals and protocols. A broad range of decoders are supported and you can write your own custom decoder using Python 3.


<img src="assets/Sigrok_Scan_I2C_1.png">


The above PulseView trace shows a portion of an I2C bus scan with stacked I2C protocol and timing decoders.

